In [1]:
import tensorflow as tf
from tensorflow import keras

In [4]:
# 加载数据
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

In [27]:
# 数据预处理
def preprocess_data(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255
    x = tf.reshape(x, [28 * 28])
    y = tf.cast(y, dtype=tf.int32)
    y = tf.one_hot(y, depth=10)
    return x, y
    

In [28]:
# 划分训练接和测试集
batch_size = 128
db_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
db_train = db_train.map(preprocess_data).shuffle(10000).batch(batch_size)

db_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))
db_test = db_test.map(preprocess_data).batch(batch_size)

In [29]:
# 验证数据类型
sample = next(iter(db_train))
print(sample[0].shape, sample[1].shape)

(128, 784) (128, 10)


In [30]:
model = keras.Sequential([
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10)
])

# [None, 28 * 28] none 代表样本数量不确定 可以是任意个样本, 样本必须是28 * 28的
model.build(input_shape=[None, 28 * 28])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             multiple                  401920    
_________________________________________________________________
dense_13 (Dense)             multiple                  131328    
_________________________________________________________________
dense_14 (Dense)             multiple                  32896     
_________________________________________________________________
dense_15 (Dense)             multiple                  8256      
_________________________________________________________________
dense_16 (Dense)             multiple                  2080      
_________________________________________________________________
dense_17 (Dense)             multiple                  330       
Total params: 576,810
Trainable params: 576,810
Non-trainable params: 0
________________________________________________

In [31]:
model.compile(optimizer=keras.optimizers.Adam(1e-3), 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True), 
              metrics=['accuracy'])


In [33]:
model.fit(db_train, 
          epochs=10,  # 训练次数
          validation_freq=1, # 验证集调用的评率
          validation_data=db_test)

Epoch 1/10
469/469 [==============================] - 8s 16ms/step - loss: 0.2718 - accuracy: 0.8995 - val_loss: 0.4060 - val_accuracy: 0.8564
Epoch 2/10
469/469 [==============================] - 7s 14ms/step - loss: 0.2565 - accuracy: 0.9035 - val_loss: 0.3838 - val_accuracy: 0.8654
Epoch 3/10
469/469 [==============================] - 6s 13ms/step - loss: 0.2426 - accuracy: 0.9097 - val_loss: 0.3838 - val_accuracy: 0.8659
Epoch 4/10
469/469 [==============================] - 7s 14ms/step - loss: 0.2334 - accuracy: 0.9129 - val_loss: 0.3762 - val_accuracy: 0.8721
Epoch 5/10
469/469 [==============================] - 8s 16ms/step - loss: 0.2253 - accuracy: 0.9154 - val_loss: 0.3717 - val_accuracy: 0.8777
Epoch 6/10
469/469 [==============================] - 7s 15ms/step - loss: 0.2152 - accuracy: 0.9201 - val_loss: 0.3802 - val_accuracy: 0.8744
Epoch 7/10
469/469 [==============================] - 7s 15ms/step - loss: 0.2055 - accuracy: 0.9220 - val_loss: 0.3840 - val_accuracy: 0.8720

In [34]:
model.evaluate(db_test)

79/79 [==============================] - 1s 12ms/step - loss: 0.3868 - accuracy: 0.8794


[0.38677560585209086, 0.8794]